In [1]:
output_node_names_list = ['time_distributed_out/Reshape_1']
output_node_names = ",".join(output_node_names_list)

In [2]:
from tensorflow.python.tools import freeze_graph
import os
from tensorflow.core.protobuf import saver_pb2

input_graph_path = os.path.join('Model/', 'input_graph.pb')
input_saver_def_path = ""
input_binary = False
restore_op_name = ""
filename_tensor_name = ""
output_graph_path = os.path.join('Model/', 'output_graph.pb')
clear_devices = False
checkpoint_path = "Model/saved_checkpoint-0"
input_meta_graph = os.path.join('Model/', "saved_checkpoint.meta")

freeze_graph.freeze_graph(
        input_graph_path,
        input_saver_def_path,
        input_binary,
        checkpoint_path,
        output_node_names,
        restore_op_name,
        filename_tensor_name,
        output_graph_path,
        clear_devices,
        "",
        "",
        input_meta_graph,
        checkpoint_version= saver_pb2.SaverDef.V2
        )
    

Instructions for updating:
Use tf.gfile.GFile.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Restoring parameters from Model/saved_checkpoint-0
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 19 variables.
INFO:tensorflow:Converted 19 variables to const ops.


In [3]:
import tensorflow as tf
import sys
from tensorflow.python.platform import gfile

from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.util import compat

with tf.Session() as sess:
	model_filename = 'Model/output_graph.pb'
	with gfile.FastGFile(model_filename, 'rb') as f:

		data = compat.as_bytes(f.read())
		sm = saved_model_pb2.SavedModel()
		sm.ParseFromString(data)
		#print(sm)
		if 1 != len(sm.meta_graphs):
			print('More than one graph found. Not sure which to write')
			sys.exit(1)
			
		#graph_def = tf.GraphDef()
		#graph_def.ParseFromString(sm.meta_graphs[0])
		g_in = tf.import_graph_def(sm.meta_graphs[0].graph_def)
	LOGDIR='summaries'

train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)
train_writer.flush()
train_writer.close()

In [32]:
import tensorflow as tf
import sys
from google.protobuf import text_format
from tensorflow.python.lib.io import file_io
from tensorflow.core.protobuf import saved_model_pb2
def summary_from_pbtxt_proof():
    with tf.Session() as sess:
        model_filename = 'Model/input_graph.pb'
        sm = saved_model_pb2.SavedModel()
        file_content = file_io.FileIO(model_filename, "rb").read()
        text_format.Merge(file_content.decode("utf-8"), sm)
        #print(sm)
        if 1 != len(sm.meta_graphs):
            print('More than one graph found. Not sure which to write')
            sys.exit(1)
            #graph_def = tf.GraphDef()
            #graph_def.ParseFromString(sm.meta_graphs[0])
            g_in = tf.import_graph_def(sm.meta_graphs[0].graph_def)
        LOGDIR='summaries'

    train_writer = tf.summary.FileWriter(LOGDIR)
    train_writer.add_graph(sess.graph)
    train_writer.flush()
    train_writer.close()
